In [1]:
import os
import typing as ty
from pathlib import Path

import laspy
import polars as pl
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import torch
from pyntcloud import PyntCloud
from sklearn.metrics import confusion_matrix

In [2]:
# Helper function to read PCD data - this needs to be implemented
def read_pcd(file_path):
    cloud = PyntCloud.from_file(
        file_path
    )
    return cloud.points

def create_pc_dataset(pc_dict: dict[str, ty.Any]) -> xr.Dataset:
    """
    Create an Xarray dataset from a point cloud dictionary.

    Args:
        pc_dict (Dict[str, Any]): A dictionary containing point cloud data with the following keys:
            - 'coord': The point coordinates as a 2D numpy array of shape (num_points, 3).
            - 'color': The point colors as a 2D numpy array of shape (num_points, 3).
            - 'normal': The point normals as a 2D numpy array of shape (num_points, 3).
            - 'semantic_gt20': The semantic labels (20 classes) as a 1D numpy array of shape (num_points,).
            - 'semantic_gt200': The semantic labels (200 classes) as a 1D numpy array of shape (num_points,).
            - 'instance_gt': The instance labels as a 1D numpy array of shape (num_points,).
            - 'scene_id': The scene ID as a string.

    Returns:
        xr.Dataset: An Xarray dataset representing the point cloud data.

    Raises:
        KeyError: If any of the required keys are missing from the input dictionary.
        ValueError: If the input arrays have inconsistent shapes or if the 'scene_id' is not a string.

    Example:
        pc_dict = {
            'coord': np.array([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]]),
            'color': np.array([[255, 0, 0], [0, 255, 0]]),
            'normal': np.array([[0.0, 0.0, 1.0], [0.0, 1.0, 0.0]]),
            'semantic_gt20': np.array([1, 2]),
            'semantic_gt200': np.array([10, 20]),
            'instance_gt': np.array([1, 1]),
            'scene_id': 'scene_0001'
        }
        pc_dataset = create_pc_dataset(pc_dict)
    """
    # Check if all required keys are present in the input dictionary
    # required_keys = [
    #     'coord', 'color', 'normal', 'semantic_gt20', 'semantic_gt200', 
    #     'instance_gt', 'scene_id'
    # ]
    required_keys = [
        'coord', 'color', 'normal', 'semantic_gt20', 'scene_id'
    ]
    for key in required_keys:
        if key not in pc_dict:
            raise KeyError(f"Missing required key '{key}' in the input dict")

    # Check if the input arrays have consistent shapes
    num_points = len(pc_dict['coord'])
    for key in ['color', 'normal']:
        if pc_dict[key].shape != (num_points, 3):
            raise ValueError(f"Array '{key}' has an inconsistent shape. Expected ({num_points}, 3).")
    for key in ['semantic_gt20']: #, 'semantic_gt200', 'instance_gt']:
        if pc_dict[key].shape != (num_points,):
            raise ValueError(f"Array '{key}' has an inconsistent shape. Expected ({num_points},).")

    # Create an Xarray dataset from the point cloud dictionary
    pc_dataset = xr.Dataset(
        data_vars=dict(
            coord=(['point', 'coord_dim'], pc_dict['coord']),
            color=(['point', 'color_dim'], pc_dict['color']),
            normal=(['point', 'normal_dim'], pc_dict['normal']),
            semantic_gt20=(['point'], pc_dict['semantic_gt20']),
            # semantic_gt200=(['point'], pc_dict['semantic_gt200']),
            # instance_gt=(['point'], pc_dict['instance_gt'])
        ),
        coords=dict(
            point=np.arange(num_points),
            coord_dim=['x', 'y', 'z'],
            color_dim=['r', 'g', 'b'],
            normal_dim=['nx', 'ny', 'nz']
        ),
        attrs=dict(scene_id=pc_dict['scene_id'])
    )

    return pc_dataset


# Define the mappings from Polars to PCD data types
POLARS_PCD_TYPE_MAPPINGS = [
    (pl.Float32, ('F', 4)),
    (pl.Float64, ('F', 8)),
    (pl.UInt8, ('U', 1)),
    (pl.UInt16, ('U', 2)),
    (pl.UInt32, ('U', 4)),
    (pl.UInt64, ('U', 8)),
    (pl.Int16, ('I', 2)),
    (pl.Int32, ('I', 4)),
    (pl.Int64, ('I', 8)),
]
POLARS_TYPE_TO_PCD_TYPE = {dtype: mapping for dtype, mapping in POLARS_PCD_TYPE_MAPPINGS}


def write_pcd(filename, data, metadata=None, batch_size=500000, binary=False):
    """
    Writes a PCD file from a Polars DataFrame or LazyFrame in batches.

    Parameters
    ----------
    filename: str
        Path to the output PCD file.
    data: Polars DataFrame or LazyFrame
        DataFrame or LazyFrame containing the point cloud data.
    metadata: dict, optional
        Dictionary containing PCD metadata. If not provided, it will be
        generated from the data.
    batch_size: int, optional
        Size of each batch to be processed.
    binary: bool, optional
        If True, writes the PCD file in binary format. Otherwise, writes in ASCII format.
    """
    # Check if data is a LazyFrame
    is_lazy = isinstance(data, pl.LazyFrame)

    # Rename columns if they exist
    # rename_map = {'X': 'x', 'Y': 'y', 'Z': 'z'}
    # data = data.rename(rename_map)

    # Fill nans for eigenentropy
    if is_lazy:
        data = data.with_columns([
            pl.col("x").cast(pl.Float32),  # open3d seems to req single precision float
            pl.col("y").cast(pl.Float32),
            pl.col("z").cast(pl.Float32)
        ])
        data = data.select(pl.exclude(pl.String))
    else:
        data = data.with_columns([
            pl.col("x").cast(pl.Float32),  # open3d seems to req single precision float
            pl.col("y").cast(pl.Float32),
            pl.col("z").cast(pl.Float32)
        ])
        data = data.select(pl.exclude(pl.Utf8))

    if metadata is None:
        n_rows = data.height if not is_lazy else data.select(pl.col("*")).count().item()
        # Generate metadata from the first batch of data
        first_batch = data.limit(batch_size).collect() if is_lazy else data.head(batch_size)
        metadata = {
            'version': '.7',
            'fields': first_batch.columns,
            'size': [POLARS_TYPE_TO_PCD_TYPE[dtype][1] for dtype in first_batch.dtypes],
            'type': [POLARS_TYPE_TO_PCD_TYPE[dtype][0] for dtype in first_batch.dtypes],
            'count': [1] * len(first_batch.columns),
            'width': n_rows,
            'height': 1,
            'viewpoint': [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
            'points': n_rows,
            'data': 'binary' if binary else 'ascii'
        }

    with open(filename, 'wb') as f:
        # Write metadata
        f.write(f"VERSION {metadata['version']}\n".encode())
        f.write(f"FIELDS {' '.join(metadata['fields'])}\n".encode())
        f.write(f"SIZE {' '.join(map(str, metadata['size']))}\n".encode())
        f.write(f"TYPE {' '.join(metadata['type'])}\n".encode())
        f.write(f"COUNT {' '.join(map(str, metadata['count']))}\n".encode())
        f.write(f"WIDTH {metadata['width']}\n".encode())
        f.write(f"HEIGHT {metadata['height']}\n".encode())
        f.write(f"VIEWPOINT {' '.join(map(str, metadata['viewpoint']))}\n".encode())
        f.write(f"POINTS {metadata['points']}\n".encode())
        f.write(f"DATA {metadata['data']}\n".encode())

        # Process and write data in batches
        for start_row in range(0, metadata["points"], batch_size):
            batch_df = data.slice(start_row, batch_size).collect() if is_lazy else data[start_row:start_row+batch_size]
            if binary:
                # Write batch in binary format
                data_buffer = batch_df.to_pandas().to_records(index=False).tobytes()
                f.write(data_buffer)
            else:
                # Write batch in ASCII format
                np.savetxt(f, batch_df.to_numpy(), fmt=' '.join(['%s'] * batch_df.width))

    return filename

def convert_pcd_to_las(
    pcd_file_path: Path,
    las_file_path: Path,
    pred_col: str = "pred",
    gt_col: str = "gt"
) -> Path:
    # Read the PCD file
    pcd_data = read_pcd(str(pcd_file_path))

    # Create a new LAS file header
    header = laspy.LasHeader(version="1.4", point_format=2)
    
    # Add extra dimensions for predicted_class and predicted_proba
    header.add_extra_dim(laspy.ExtraBytesParams(name="predicted_class", type="f"))
    header.add_extra_dim(laspy.ExtraBytesParams(name="gt_class", type="f"))
    
    # Create an empty LAS file with the header
    las = laspy.LasData(header)

    # Fill the LAS file with data
    las.x, las.y, las.z = pcd_data['x'], pcd_data['y'], pcd_data['z']
    if "intensity" in pcd_data.columns:
        las.intensity = pcd_data['intensity']
    las.red, las.green, las.blue = pcd_data["r"], pcd_data["g"], pcd_data["b"]

    # Set values for the extra dimensions
    las.predicted_class = pcd_data[pred_col]
    las.gt_class = pcd_data[gt_col]

    # Write the LAS file to disk
    las.write(las_file_path)
    return las_file_path


Load the scenes and the predictions

In [10]:
repo_root = Path("..")

scenes_dir = Path(
    "/data/sdd/qh/vox6res0.2/val"
)
results_dir = Path(
    "/data/sdd/results_qh/vox6res0.2/result"
)

predictions = {
    path.stem.rstrip("_pred"): np.load(path) 
    for path in results_dir.glob("*.npy")
}
scenes = {
    path.stem: torch.load(path) 
    for path in scenes_dir.glob("*.pth")
}

In [11]:
classes = [
    'wall', 'floor', 'cabinet', 'bed', 'chair', 'sofa', 'table', 'door',
    'window', 'bookshelf', 'picture', 'counter', 'desk', 'curtain',
    'refridgerator', 'shower curtain', 'toilet', 'sink', 'bathtub',
    'otherfurniture'
]

Write everything out

In [12]:
for scene_id, pred in predictions.items():
    points = scenes[scene_id]
    labels = points["semantic_gt20"]
    ds = create_pc_dataset(points)
    ds = ds.assign_coords(classes_scannet=xr.DataArray(np.array(classes), dims=["classes_scannet"]))
    ds["pred"] =(("point",), pred)
    ds["color"] = ds["color"].astype(np.uint8)
    df = pd.concat(
        [
            ds["coord"].to_pandas(),
            ds["color"].to_pandas(),
            ds["pred"].to_pandas().rename("pred"),
            ds["semantic_gt20"].to_pandas().rename("gt")
        ],
        axis=1
    )
    lf = pl.from_pandas(df)
    pcd_out = f"{scene_id}.pcd"
    las_out = f"{scene_id}.las"
    write_pcd(pcd_out, lf, binary=True)
    print(f"Write: {las_out}")
    convert_pcd_to_las(pcd_out, las_out)
    os.remove(pcd_out)

Write: voxel_0_2_0.las
Write: voxel_0_2_1.las
Write: voxel_0_2_2.las
Write: voxel_0_2_3.las
Write: voxel_0_2_4.las
Write: voxel_0_3_0.las
Write: voxel_0_3_1.las
Write: voxel_0_3_2.las
Write: voxel_0_3_3.las
Write: voxel_0_3_4.las
Write: voxel_1_0_0.las
Write: voxel_1_0_1.las
Write: voxel_1_0_2.las
Write: voxel_1_0_3.las
Write: voxel_1_2_0.las
Write: voxel_1_2_1.las
Write: voxel_1_2_2.las
Write: voxel_1_2_3.las
Write: voxel_1_2_4.las
Write: voxel_1_3_0.las
Write: voxel_1_3_1.las
Write: voxel_1_3_2.las
Write: voxel_1_3_3.las
Write: voxel_1_3_4.las
Write: voxel_2_0_0.las
Write: voxel_2_0_1.las
Write: voxel_2_0_2.las
Write: voxel_2_0_3.las
Write: voxel_2_0_4.las
Write: voxel_2_2_1.las
Write: voxel_2_2_2.las
Write: voxel_2_2_3.las
Write: voxel_2_2_4.las
Write: voxel_2_2_5.las
Write: voxel_2_3_0.las
Write: voxel_2_3_1.las
Write: voxel_2_3_2.las
Write: voxel_2_3_3.las
Write: voxel_2_3_4.las
Write: voxel_2_3_5.las
Write: voxel_2_4_0.las
Write: voxel_2_4_1.las
Write: voxel_3_2_0.las
Write: voxe

Confusion matrix stuff

need to figure out labelling first

In [6]:
if False:
    df = pd.DataFrame(dict(pred=pred, gt=labels))
    
    # Extract the predicted and ground truth labels from the DataFrame
    y_pred = df['pred']
    y_true = df['gt']
    
    # Compute the confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(12, 10))  # Adjust the figure size as needed
    
    # Plot the confusion matrix
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    
    # Add a color bar
    cbar = ax.figure.colorbar(im, ax=ax)
    
    # Define the class names
    #classes = class_name
    classes = [
        'wall', 'floor', 'cabinet', 'bed', 'chair', 'sofa', 'table', 'door',
        'window', 'bookshelf', 'picture', 'counter', 'desk', 'curtain',
        'refridgerator', 'shower curtain', 'toilet', 'sink', 'bathtub',
        'otherfurniture'
    ]
    
    # Set the tick labels and positions
    ax.set(xticks=np.arange(len(classes)),
           yticks=np.arange(len(classes)),
           xticklabels=classes, yticklabels=classes,
           xlabel='Predicted Labels',
           ylabel='True Labels')
    
    # Rotate the tick labels and set their alignment
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    
    # Loop over data dimensions and create text annotations
    fmt = 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    
    # Set the title
    ax.set_title("Confusion Matrix")
    
    # Adjust the layout and display the plot
    fig.tight_layout()
    plt.show()

In [7]:
import numpy as np
import pandas as pd
import os

def npy_to_series(directory):
    """
    Load all .npy files from the specified directory and collapse them into a single pandas Series.
    
    Args:
    directory (str): The path to the directory containing .npy files.
    
    Returns:
    pd.Series: A Pandas Series containing all elements from the .npy files.
    """
    series_list = []  # Initialize an empty list to hold arrays from each file
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.npy'):
            file_path = os.path.join(directory, filename)
            data = np.load(file_path)  # Load .npy file
            series_list.append(pd.Series(data))  # Append data as a pandas Series to the list
            
    # Concatenate all series in the list into a single Series
    full_series = pd.concat(series_list, ignore_index=True)
    
    return full_series

# Example usage
# Assuming your .npy files are stored in 'path_to_your_directory'
result_series = npy_to_series(results_dir)
print(result_series)


ValueError: No objects to concatenate

In [ ]:
result_series.value_counts()

In [ ]:
result_series.value_counts(normalize=True)
